
# 🌀 Poseidon semplificato — Implementazione didattica in Python

Questa è una versione semplificata dell'algoritmo Poseidon, pensata per aiutarti a capire **ogni singolo passaggio** della funzione di hash, soprattutto per analisi **differenziali** e **lineari**.

🔍 Qui vedrai:
- Stato dopo ogni round
- S-box
- Matrice MDS
- Costanti di round
- Campo finito

Tutto è spiegato e visualizzabile. Puoi modificarlo a piacere per testare ipotesi o fare esperimenti di crittoanalisi.


In [1]:

# ⚙️ Parametri principali

# Campo finito piccolo (debug-friendly)
p = 2**31 - 1  # Primo grande abbastanza per test ma semplice

# Numero di elementi nello stato (t=3)
t = 3

# Numero di round
full_rounds = 4
partial_rounds = 4
n_rounds = full_rounds + partial_rounds

# Esponente della S-box
alpha = 5


In [2]:

# 📏 Costanti di round (esempio: 8 round x 3 elementi = 24 costanti)
round_constants = [(i * 17 + 42) % p for i in range(n_rounds * t)]

# 🧮 Matrice MDS fissa 3x3 (esempio didattico)
MDS = [
    [2, 3, 1],
    [1, 2, 3],
    [3, 1, 2]
]


In [3]:

# 🔢 Funzione S-box
def sbox(x):
    return pow(x, alpha, p)

# ♻️ Aggiunta costanti di round
def add_constants(state, round_index):
    return [(x + round_constants[round_index * t + i]) % p for i, x in enumerate(state)]

# 🔄 Moltiplicazione per matrice MDS
def mix(state):
    return [
        sum(state[j] * MDS[i][j] for j in range(t)) % p
        for i in range(t)
    ]


In [4]:

# 🌀 Permutazione Poseidon
def poseidon_permutation(input_state):
    state = input_state[:]
    for r in range(n_rounds):
        print(f"🔁 Round {r}")
        
        # Add round constants
        state = add_constants(state, r)
        print("  ➕ After constants:", state)
        
        # Apply S-box
        if r < full_rounds // 2 or r >= n_rounds - full_rounds // 2:
            # Full round: tutte le S-box
            state = [sbox(x) for x in state]
        else:
            # Partial round: solo la prima S-box
            state = [sbox(state[0])] + state[1:]
        print("  🧮 After S-box:", state)
        
        # MDS mixing
        state = mix(state)
        print("  🔗 After MDS:", state)
        
    return state


In [5]:

# ▶️ Esempio di test

input_state = [1, 2, 3]  # Input di esempio

print("📥 Stato iniziale:", input_state)
output_state = poseidon_permutation(input_state)
print("\n📤 Stato finale:", output_state)


📥 Stato iniziale: [1, 2, 3]
🔁 Round 0
  ➕ After constants: [43, 61, 79]
  🧮 After S-box: [147008443, 844596301, 929572752]
  🔗 After MDS: [1609894894, 329952007, 997283487]
🔁 Round 1
  ➕ After constants: [1609894987, 329952117, 997283614]
  🧮 After S-box: [1122066081, 140595410, 1005422098]
  🔗 After MDS: [1523856843, 124555901, 1222670555]
🔁 Round 2
  ➕ After constants: [1523856987, 124556062, 1222670733]
  🧮 After S-box: [488245171, 124556062, 1222670733]
  🔗 After MDS: [425345614, 110402200, 1887149394]
🔁 Round 3
  ➕ After constants: [425345809, 110402412, 1887149623]
  🧮 After S-box: [1488630649, 110402412, 1887149623]
  🔗 After MDS: [900650863, 928433401, 1908142664]
🔁 Round 4
  ➕ After constants: [900651109, 928433664, 1908142944]
  🧮 After S-box: [725002412, 928433664, 1908142944]
  🔗 After MDS: [1848481466, 1863847631, 477275847]
🔁 Round 5
  ➕ After constants: [1848481763, 1863847945, 477276178]
  🧮 After S-box: [1474212368, 1863847945, 477276178]
  🔗 After MDS: [427310161, 191

In [6]:

# 🧪 Esperimento differenziale
# Input modificato con una piccola differenza

input_A = [1, 2, 3]
input_B = [1, 2, 4]  # Differenza su ultimo elemento

print("📥 Input A:", input_A)
out_A = poseidon_permutation(input_A)
print("\n📥 Input B:", input_B)
out_B = poseidon_permutation(input_B)

# Differenza in output
diff = [(a - b) % p for a, b in zip(out_A, out_B)]
print("\n🔬 Differenza in output:", diff)


📥 Input A: [1, 2, 3]
🔁 Round 0
  ➕ After constants: [43, 61, 79]
  🧮 After S-box: [147008443, 844596301, 929572752]
  🔗 After MDS: [1609894894, 329952007, 997283487]
🔁 Round 1
  ➕ After constants: [1609894987, 329952117, 997283614]
  🧮 After S-box: [1122066081, 140595410, 1005422098]
  🔗 After MDS: [1523856843, 124555901, 1222670555]
🔁 Round 2
  ➕ After constants: [1523856987, 124556062, 1222670733]
  🧮 After S-box: [488245171, 124556062, 1222670733]
  🔗 After MDS: [425345614, 110402200, 1887149394]
🔁 Round 3
  ➕ After constants: [425345809, 110402412, 1887149623]
  🧮 After S-box: [1488630649, 110402412, 1887149623]
  🔗 After MDS: [900650863, 928433401, 1908142664]
🔁 Round 4
  ➕ After constants: [900651109, 928433664, 1908142944]
  🧮 After S-box: [725002412, 928433664, 1908142944]
  🔗 After MDS: [1848481466, 1863847631, 477275847]
🔁 Round 5
  ➕ After constants: [1848481763, 1863847945, 477276178]
  🧮 After S-box: [1474212368, 1863847945, 477276178]
  🔗 After MDS: [427310161, 191285851,


## 🔍 Analisi Differenziale

In questa sezione esaminiamo come piccole differenze nell'input si propagano attraverso i round.


In [7]:

import matplotlib.pyplot as plt

def count_bit_differences(a, b):
    return sum(bin(x ^ y).count("1") for x, y in zip(a, b))

# Test con più input differenziali
base_input = [1, 2, 3]
differences = [1, 2, 4, 8, 16]
bit_diffs = []

for d in differences:
    mod_input = [1, 2, 3 + d]
    out_base = poseidon_permutation(base_input)
    out_mod = poseidon_permutation(mod_input)
    bit_diff = count_bit_differences(out_base, out_mod)
    bit_diffs.append(bit_diff)

plt.figure(figsize=(8, 4))
plt.plot(differences, bit_diffs, marker='o')
plt.title("Analisi Differenziale: differenze bit in output")
plt.xlabel("Differenza applicata a input[2]")
plt.ylabel("Bit diversi nell'output")
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'matplotlib'


## 🧮 Analisi Lineare (semplificata)

Studiamo la relazione lineare tra input e output per osservare eventuali pattern lineari.


In [ ]:

import numpy as np

# Usiamo una versione linearizzata per test (disattiviamo la S-box)
def poseidon_linearized(input_state):
    state = input_state[:]
    for r in range(n_rounds):
        state = add_constants(state, r)
        # Non applichiamo la S-box (x^5) per analisi lineare
        state = mix(state)
    return state

# Genera input casuali e verifica output linearizzati
inputs = [[i, i+1, i+2] for i in range(20)]
outputs = [poseidon_linearized(inp) for inp in inputs]

# Costruzione matrice input-output
input_matrix = np.array(inputs)
output_matrix = np.array(outputs)

plt.figure(figsize=(8, 5))
plt.imshow(output_matrix % 2, cmap="gray", aspect="auto")
plt.title("Pattern dei bit in output (mod 2)")
plt.xlabel("Elemento output")
plt.ylabel("Esempio di input")
plt.colorbar(label="Bit")
plt.show()
